# **バンディットアルゴリズムの理論限界**

今回は私の勉強のためにもバンディットアルゴリズムの**理論限界**についてノートブックで説明していきたいと思います．

### 理論限界

バンディットアルゴリズムでは**最適な報酬**を得るために, 探索を必要とするのですが, ある一定の探索回数にに満たないときに, 最適なアームを選ぶ方針を導き出せないとされています．

***例***: *アーム1は10%の確率で報酬**1**を出し, アーム2は1%の確率で報酬**100**を出すとします.この時, アームを2を引くのが最適な方針なのですが, アーム2から報酬を出すには平均で100回の試行が必要なため, **10回**などの探索で辞めてしまったら最適な方針を導けませんね.*


つまり, **最適な方針をどのくらい最低限の試行**で導けるのかが**理論限界**の意味するところです.

##### ここでコードで例を見てみましょう. 今回は別のノートブックで説明したETCアルゴリズムで確認.

In [1]:
import numpy as np
p = np.array([0.1,0.01]) #各アームの報酬が出る確率
r = np.array([1,100]) #各アームの報酬
cul_reward = np.zeros(2)

class Theory_bandit:
    def __init__(self,t):
       self.p = np.array([0.1,0.01]) #各アームの報酬が出る確率
       self.r = np.array([1,100]) #各アームの報酬
       self.cul_reward = np.zeros(2)  #累積報酬
       self.choice_arm = np.zeros(2) #選択総回数
       self.t = t #各アームの探索数
    
    def get_reward(self,index):
        reward = self.r[index] if np.random.rand() < self.p[index] else 0
        return reward

    def update(self,index,reward):
        self.cul_reward[index] += reward
        self.choice_arm[index] += 1

    def act(self,round):
        if round < 2 * self.t: #決められた探索回数まで0か1を引く続ける
            if round < self.t: 
                return 0

            elif round < 2 * self.t:
                return 1

        else:
            return np.argmax(self.cul_reward)

    def simulate(self,T):
        for round in range(T):
            arm = self.act(round)
            reward = self.get_reward(arm)
            self.update(arm,reward)


上の例で示したモデルがこのようなコードで書けました.


次に各アームをそれぞれ10回, 100回探索した時の選択されたアームの回数を確認しましょう.

In [2]:
agent = Theory_bandit(10)
agent.simulate(1000)

In [3]:
print(f'10回の探索でそれぞれのアームが試行された回数はアーム0が{agent.choice_arm[0]}回, アーム1が{agent.choice_arm[1]}回です.')

10回の探索でそれぞれのアームが試行された回数はアーム0が990.0回, アーム1が10.0回です.


In [4]:
agent = Theory_bandit(100)
agent.simulate(1000)

In [5]:
print(f'100回の探索でそれぞれのアームが試行された回数はアーム0が{agent.choice_arm[0]}, アーム1が{agent.choice_arm[1]}です.')

100回の探索でそれぞれのアームが試行された回数はアーム0が100.0, アーム1が900.0です.


このように10回の探索では最適な方針を導けないことがわかりましたね.

単純に探索回数を減らすのではなく, 最適な方針を導ける且つ, 最低限の探索を目指すのがバンディットアルゴリズムということがわかります．

---

ここで最適な方針を導くための条件として**一貫性**という定理があります．

### 一貫性

*ある方策が一貫性を持つとは, 任意の固定した $a>0$ と真の確率分布の組 $({P_{i}})^K_{i=1} \in P^k$ に対して, その方針のもとで $\mathbb{E}[regret(T)] = \mathrm{o}(T^a)$ が成り立つことを言う.*
